In [1]:
import numpy as np

In [2]:
from skimage.graph import cut_normalized

In [3]:
from skimage.io import imread

In [4]:
import matplotlib.pyplot as plt

In [5]:
from PIL import Image

In [6]:
import scipy.sparse as sp

In [7]:
def foreground_background_segmentation(image_path):
    # Load the input image
    image = imread(image_path)

    # Graph Construction
    graph, node_indices = construct_graph(image)

    # Minimum Cut Computation
    minimum_cut = find_minimum_cut(graph)

    # Label Assignment
    segmentation = assign_labels(minimum_cut, node_indices)

    return segmentation

In [8]:

def construct_graph(image):
    height, width = image.shape[:2]
    num_nodes = height * width

    # Create lists to store the row, column, and weight values
    rows = []
    cols = []
    weights = []

    # Map node indices to (i, j) pixel coordinates
    node_indices = np.arange(num_nodes).reshape(height, width)

    # Add edges between neighboring pixels
    for i in range(height):
        for j in range(width):
            current_node = node_indices[i, j]

            # Check and connect with neighboring pixels
            if i > 0:
                rows.append(current_node)
                cols.append(node_indices[i-1, j])
                weights.append(calculate_edge_weight(image[i, j], image[i-1, j]))
            if j > 0:
                rows.append(current_node)
                cols.append(node_indices[i, j-1])
                weights.append(calculate_edge_weight(image[i, j], image[i, j-1]))
            if i < height - 1:
                rows.append(current_node)
                cols.append(node_indices[i+1, j])
                weights.append(calculate_edge_weight(image[i, j], image[i+1, j]))
            if j < width - 1:
                rows.append(current_node)
                cols.append(node_indices[i, j+1])
                weights.append(calculate_edge_weight(image[i, j], image[i, j+1]))

    # Convert the lists to NumPy arrays
    rows = np.array(rows)
    cols = np.array(cols)
    weights = np.array(weights)

    # Create the sparse graph in COO format
    graph = sp.coo_matrix((weights, (rows, cols)), shape=(num_nodes, num_nodes)).tocsr()

    return graph, node_indices


In [9]:
def calculate_edge_weight(pixel1, pixel2):
    # Calculate the dissimilarity or similarity between two pixels
    # based on color, texture, or other visual features
    # Return the weight of the edge between the two pixels
    return np.abs(pixel1 - pixel2)  # Example: using absolute color difference

In [10]:
def find_minimum_cut(graph):
    # Apply a minimum cut algorithm to compute the minimum cut
    # Return the minimum cut, which represents the optimal partitioning
    # of the graph into foreground and background regions
    _, minimum_cut = cut_normalized(graph)
    return minimum_cut

In [11]:
def assign_labels(minimum_cut, node_indices):
    height, width = node_indices.shape
    segmentation = np.zeros((height, width))

    # Assign labels based on the minimum cut result
    foreground_indices = np.where(minimum_cut == 1)[0]
    foreground_pixels = node_indices.flatten()[foreground_indices]
    segmentation.flat[foreground_pixels] = 1

    return segmentation

In [12]:
# Custom input image path
image_path = r'C:\Users\rohit\Downloads\Screenshot 2023-05-29 023446.png'

# Perform foreground/background segmentation
segmentation_result = foreground_background_segmentation(image_path)

ValueError: row, column, and data arrays must be 1-D